In [1]:
from detectron2.data.datasets import register_coco_instances
from detectron2.data import DatasetCatalog, MetadataCatalog
keypoint_names = ["front"]
keypoint_flip_map = []

register_coco_instances("merged_train", {}, r"D:\models\detectron2\datasets\merged\annotations\train.json", r"D:\models\detectron2\datasets\merged\images")
register_coco_instances("merged_val", {}, r"D:\models\detectron2\datasets\merged\annotations\val.json", r"D:\models\detectron2\datasets\merged\images")
MetadataCatalog.get("merged_train").set(keypoint_names=keypoint_names, keypoint_flip_map=keypoint_flip_map)
MetadataCatalog.get("merged_val").set(keypoint_names=keypoint_names, keypoint_flip_map=keypoint_flip_map)

register_coco_instances("merged_oc_train", {}, r"D:\models\detectron2\datasets\merged_oc\annotations\train.json", r"D:\models\detectron2\datasets\merged_oc\images")
register_coco_instances("merged_oc_val", {}, r"D:\models\detectron2\datasets\merged_oc\annotations\val.json", r"D:\models\detectron2\datasets\merged_oc\images")
MetadataCatalog.get("merged_oc_train").set(keypoint_names=keypoint_names, keypoint_flip_map=keypoint_flip_map)
MetadataCatalog.get("merged_oc_val").set(keypoint_names=keypoint_names, keypoint_flip_map=keypoint_flip_map)

evaluation_dataset_path = r"D:\models\detectron2\datasets\testing"

In [13]:
from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.engine import DefaultTrainer,DefaultPredictor
from detectron2.evaluation import COCOEvaluator
from detectron2.utils.visualizer import Visualizer
from detectron2.data.transforms import RandomFlip, RandomCrop, RandomRotation, RandomBrightness, RandomContrast
from detectron2.data import DatasetMapper, build_detection_train_loader
import torch
import cv2
import os

output_dir = r"D:\models\detectron2\manipulator\evaluation"
dataset = "merged"

def eval(cfg):
    cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
    predictor = DefaultPredictor(cfg)
    for root, dirs, files in os.walk(evaluation_dataset_path):
        folder = root.split('/')[-1]
        if not os.path.exists(os.path.join(os.path.join(cfg.OUTPUT_DIR, "inference"), folder)):
            os.makedirs(os.path.join(os.path.join(cfg.OUTPUT_DIR, "inference"), folder))
        for image in files:
            print(image)
            img = cv2.imread(os.path.join(root, image))
            outputs = predictor(img)
            v = Visualizer(img[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
            v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
            out = v.get_image()[:, :, ::-1]
            cv2.imwrite(os.path.join(os.path.join(cfg.OUTPUT_DIR, "inference"), folder, image), out)


In [21]:
import yaml
cfg = get_cfg()
# load file as io stream
with open(r"D:\models\detectron2\manipulator\evaluation\maskeyrcnn-r50-3x-lr0.02-it270000-oc\config.yaml", 'r') as stream:
    cfg._load_cfg_from_file(stream)
cfg.OUTPUT_DIR = r"D:\models\detectron2\manipulator\evaluation\maskeyrcnn-r50-3x-lr0.02-it270000-oc"
eval(cfg)

Skip loading parameter 'proposal_generator.rpn_head.conv.weight' to the model due to incompatible shapes: (256, 256, 3, 3) in the checkpoint but (1024, 1024, 3, 3) in the model! You might want to double check if this is expected.
Skip loading parameter 'proposal_generator.rpn_head.conv.bias' to the model due to incompatible shapes: (256,) in the checkpoint but (1024,) in the model! You might want to double check if this is expected.
Skip loading parameter 'proposal_generator.rpn_head.objectness_logits.weight' to the model due to incompatible shapes: (3, 256, 1, 1) in the checkpoint but (15, 1024, 1, 1) in the model! You might want to double check if this is expected.
Skip loading parameter 'proposal_generator.rpn_head.objectness_logits.bias' to the model due to incompatible shapes: (3,) in the checkpoint but (15,) in the model! You might want to double check if this is expected.
Skip loading parameter 'proposal_generator.rpn_head.anchor_deltas.weight' to the model due to incompatible s